In [1]:
import tensorflow as tf
import keras as tfk
from keras import layers
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from numpy import array, argmax
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

print("Using TensorFlow version", tf.__version__)

seed = 420
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
print("RNG seed: ",seed)

2022-07-29 16:55:45.726521: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Using TensorFlow version 2.9.1
RNG seed:  420


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
# pathName = '/home/npasini1/Desktop/Recordings/csv/separate'
pathName = '/home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled'
numFiles = []

timestep = 5
stride = 1
n_features = 17
sx_n_gestures = 2
dx_n_gestures = 3
# sx_dataset_tot = np.empty((0,timestep,n_features))
# dx_dataset_tot = np.empty((0,timestep,n_features))
# sx_dataset_tot = []
# dx_dataset_tot = []
# sx_dataset_gesture_array_tot = np.empty((0))
# dx_dataset_gesture_array_tot = np.empty((0))

users = []
# users = []
# users = []
# users = []
users_dataset_sx = []
users_dataset_dx = []
sx_encoded_list =[]
dx_encoded_list =[]

for i in range(9):
    delete = users[i]
    sx_dataset_tot = np.empty((0,timestep,n_features))
    dx_dataset_tot = np.empty((0,timestep,n_features))
    sx_dataset_gesture_array_tot = np.empty((0))
    dx_dataset_gesture_array_tot = np.empty((0))

    fileNames = os.listdir(pathName)
    for fileName in fileNames:
        if not fileName.endswith(".csv") or delete in fileName:
            continue
        # if fileName.endswith(".csv"):
        numFiles.append(fileName)
        path = os.path.join(pathName, fileName)
        print('Now reading file: ', path)
        dataframe = pd.read_csv(path)

        #encoding
        sx_gesture = dataframe['SX Gesture']
        dx_gesture = dataframe['DX Gesture']
        sx_gesture_array = array(sx_gesture)    #numpy array
        dx_gesture_array = array(dx_gesture)    #numpy array

        df = dataframe.drop(['SX Gesture','DX Gesture','Timestamps'], axis=1)
        # df = pd.DataFrame(df)
        # df.head(5)

        samples = df.shape[0]
        # n_features = df.shape[1]

        sx_dataset = np.empty((0,timestep,n_features))
        dx_dataset = np.empty((0,timestep,n_features))
        df_copy = df.copy()
        # print(type(samples),type(timestep))
        for idx in range(samples-timestep+1):
            for j in range(timestep):
                df_copy.iloc[idx+j,[0,1,2,7,8,9]] = df.iloc[idx+j, [0,1,2,7,8,9]].astype(np.float64) - df.iloc[idx, [0,1,2,7,8,9]].astype(np.float64)
            sx_dataset = np.append(sx_dataset, np.expand_dims(df_copy.iloc[idx:idx+timestep, [7,8,9,10,11,12,13,20,21,22,23,24,25,29,30,31,32]], axis=0),axis=0)
            dx_dataset = np.append(dx_dataset, np.expand_dims(df_copy.iloc[idx:idx+timestep, [0,1,2,3,4,5,6,14,15,16,17,18,19,26,27,28,32]], axis=0),axis=0)
            j=0
        
        sx_dataset_gesture_array = sx_gesture_array[timestep-1:]
        dx_dataset_gesture_array = dx_gesture_array[timestep-1:]
        sx_dataset_tot = np.append(sx_dataset_tot,sx_dataset, axis=0)
        dx_dataset_tot = np.append(dx_dataset_tot,dx_dataset, axis=0)
        # sx_dataset_tot.append(sx_dataset)
        # dx_dataset_tot.append(dx_dataset)
        sx_dataset_gesture_array_tot = np.append(sx_dataset_gesture_array_tot, sx_dataset_gesture_array, axis=0)
        dx_dataset_gesture_array_tot = np.append(dx_dataset_gesture_array_tot, dx_dataset_gesture_array, axis=0)
        print(sx_dataset.shape)
        print(sx_dataset_tot.shape)
        # print(len(sx_dataset_tot))
        print(sx_dataset_gesture_array.shape)
        print(sx_dataset_gesture_array_tot.shape)
        print("-#-#-#-#-#-#-#-#-#-#-#-#-")
        print(dx_dataset.shape)
        print(dx_dataset_tot.shape)
        # print(len(dx_dataset_tot))
        print(dx_dataset_gesture_array.shape)
        print(dx_dataset_gesture_array_tot.shape)


    label_encoder = LabelEncoder()
    sx_integer_encoded = label_encoder.fit_transform(sx_dataset_gesture_array_tot)
    dx_integer_encoded = label_encoder.fit_transform(dx_dataset_gesture_array_tot)
    print(sx_integer_encoded)
    print(dx_integer_encoded)

    sx_encoded = to_categorical(sx_integer_encoded)
    dx_encoded = to_categorical(dx_integer_encoded)
    print(sx_encoded)
    print(dx_encoded)

    users_dataset_sx.append(sx_dataset_tot)
    users_dataset_dx.append(dx_dataset_tot)
    sx_encoded_list.append(sx_encoded)
    dx_encoded_list.append(dx_encoded)
    np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_" + delete + "_dx_5.npy", dx_dataset_tot)
    np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_" + delete + "_dx_5.npy", dx_encoded)
    np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_" + delete + "_sx_5.npy", sx_dataset_tot)
    np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_" + delete + "_sx_5.npy", sx_encoded)


Now reading file:  /home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled/AnChi2.csv
(9750, 5, 17)
(9750, 5, 17)
(9750,)
(9750,)
-#-#-#-#-#-#-#-#-#-#-#-#-
(9750, 5, 17)
(9750, 5, 17)
(9750,)
(9750,)
Now reading file:  /home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled/Mojtaba1.csv
(2589, 5, 17)
(12339, 5, 17)
(2589,)
(12339,)
-#-#-#-#-#-#-#-#-#-#-#-#-
(2589, 5, 17)
(12339, 5, 17)
(2589,)
(12339,)
Now reading file:  /home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled/Aabhas2.csv
(7865, 5, 17)
(20204, 5, 17)
(7865,)
(20204,)
-#-#-#-#-#-#-#-#-#-#-#-#-
(7865, 5, 17)
(20204, 5, 17)
(7865,)
(20204,)
Now reading file:  /home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled/Hisashi1.csv
(4096, 5, 17)
(24300, 5, 17)
(4096,)
(24300,)
-#-#-#-#-#-#-#-#-#-#-#-#-
(4096, 5, 17)
(24300, 5, 17)
(4096,)
(24300,)
Now reading file:  /home/npasini1/Desktop/dVRK_UserStudy/dVRK_pyrecorder/labeled/AnChi1.csv


KeyboardInterrupt: 

In [ ]:
# for i in range(len(dx_dataset_gesture_array_tot)):
#     if dx_dataset_gesture_array_tot[i] != 'NH' and dx_dataset_gesture_array_tot[i] != 'TB' and dx_dataset_gesture_array_tot[i] != 'NP':
#         print(i)


In [ ]:
# CHANGE NAME EVERY TIME

# np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_UserStudy_dx_5.npy", dx_dataset_tot)
# np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_UserStudy_dx_5.npy", dx_encoded)
# np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_UserStudy_sx_5.npy", sx_dataset_tot)
# np.save("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_UserStudy_sx_5.npy", sx_encoded)

In [ ]:
# CHANGE NAME EVERY TIME

# sx_dataset_tot = np.load("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_UserStudy_sx_5.npy")
# sx_encoded = np.load("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_UserStudy_sx_5.npy")
# dx_dataset_tot = np.load("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/dataset_UserStudy_dx_5.npy")
# dx_encoded = np.load("/home/npasini1/Desktop/dVRK_UserStudy/Datasets/labels_UserStudy_dx_5.npy")

In [ ]:
timestep = 5
stride = 1
n_features = 17
sx_n_gestures = 2
dx_n_gestures = 3
input_shape = (5,17)

In [ ]:
# sx_X_train, sx_X_test, sx_y_train, sx_y_test = train_test_split(sx_dataset_tot, sx_encoded, test_size=0.10, shuffle = True)
# dx_X_train, dx_X_test, dx_y_train, dx_y_test = train_test_split(dx_dataset_tot, dx_encoded, test_size=0.10, shuffle = True)
# print("Train shape: ", sx_X_train.shape)
# print("Test shape: ", sx_X_test.shape)

# input_shape = sx_X_train.shape[1:]
# sx_output_shape = sx_y_train.shape[1:]
# dx_output_shape = dx_y_train.shape[1:]
# print("Input shape: ", input_shape)
# print("SX Output shape: ", sx_output_shape)
# print("DX Output shape: ", dx_output_shape)

In [ ]:
# dataframe = pd.read_csv('/home/npasini1/Desktop/Recordings/nic_suture_10.csv')
# dataframe = pd.read_csv('/home/npasini1/Desktop/Recordings/new/Nicolo_1.csv')
# dataframe.columns

In [ ]:
# one hot encoding of Gestures

# gesture = dataframe['Predicted Gesture']
# values = array(gesture)
# print(values)

# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)

# encoded = to_categorical(integer_encoded)
# print(encoded)

In [ ]:
# print(values)

In [ ]:
# print(integer_encoded[793])
# print(values[793])

In [ ]:
# Data normalization

# labels = encoded
# df = dataframe.drop(['Predicted Gesture','Timestamps'], axis=1)

# min_max_scaler = preprocessing.MinMaxScaler()
# df_scaled = min_max_scaler.fit_transform(df)
# df_scaled = pd.DataFrame(df_scaled)
# df_scaled.head

# df = pd.DataFrame(df)

In [ ]:
# print(df.shape)

In [ ]:
# samples = df.shape[0]
# n_features = df.shape[1]
# timestep = 30
# stride = 1

# dataset = np.empty((0,timestep,n_features))
# df_copy = df
# np_scaled = np.array(df_scaled)
# print(dataset.shape)

# CON PREPROCESSING DEI DATI: MINMAX SCALER

# for idx in range(samples-timestep):
#     dataset = np.append(dataset, np.expand_dims(df_scaled[idx:idx+timestep], axis=0),axis=0)
#     # print(idx)

# SENZA PREPROCESSING DEI DATI


# for idx in range(samples-timestep):
#     for j in range(timestep):
#         df_copy.iloc[idx+j,[0,1,2,7,8,9]] = df.iloc[idx+j, [0,1,2,7,8,9]] - df.iloc[idx, [0,1,2,7,8,9]]
#     dataset = np.append(dataset, np.expand_dims(df_copy[idx:idx+timestep], axis=0),axis=0)
#     j=0

# print(dataset.shape)
# dataset_labels = labels[timestep:]


In [ ]:
# dataset_labels = labels[timestep:]
# print(dataset_labels.shape)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size=0.10, shuffle = True)
# print("Train shape: ", X_train.shape)
# print("Test shape: ", X_test.shape)

# input_shape = X_train.shape[1:]
# output_shape = y_train.shape[1:]
# print("Input shape: ", input_shape)
# print("Output shape: ", output_shape)

In [ ]:
import tensorflow as tf
import keras as tfk
import keras.layers as tfkl
import scipy.io
import os, glob, sys, pickle, time, math, gc
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model, load_model
from keras import optimizers
from keras.utils import plot_model
from keras import layers #Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Masking, BatchNormalization, Flatten, Input, Conv2D, MaxPooling1D, Conv1D, Reshape, GRU
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from keras import backend as K
from datetime import datetime

def buildModelv2(timestep, n_gestures):
    """
    An lstm-encoder model followed by dense layers, similar performance to just lstm
    """
    # lstm_hidden1 = 128 #*4
    # lstm_hidden2 = 64
    # dense_hidden1 = 64
    # output_layer = 15

    model_input = layers.Input(shape=input_shape)
    lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=tfk.regularizers.l1(l=0.001), return_sequences=True)(model_input) #previously 96
    dropout_output = layers.Dropout(rate=0.2)(lstm_output) #previously 0.4
    batch_norm1 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    lstm_output2 = layers.LSTM(64, input_shape=input_shape, kernel_regularizer=tfk.regularizers.l1(l=0.001), return_sequences=False)(batch_norm1)
    dropout_output = layers.Dropout(rate = 0.3)(lstm_output2)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    repeat_vector = layers.RepeatVector(timestep)(batch_norm2)

    flatten_output = layers.Flatten()(repeat_vector)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(flatten_output)
    dropout_output = layers.Dropout(rate = 0.2)(batch_norm2)
    dense_output1 = layers.Dense(64, activation='relu')(dropout_output)
    batch_norm3 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dense_output1)
    dense_output2 = layers.Dense(n_gestures, activation='softmax')(batch_norm3)
    lstm_classifier = Model(inputs=model_input, outputs=dense_output2)
    
    #Compile the model
    lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(1e-3), metrics=[tfk.metrics.categorical_accuracy])

    return lstm_classifier

# def buildModelv1(timesteps, n_features):
#     """
#     Simple model which yields a high accuracy of 90% on training data but sucks on the validation data
#     """
#     model_input = layers.Input(shape=input_shape)
#     lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=tfk.regularizers.l1(l=0.001), return_sequences=True)(model_input)
#     dropout_output = layers.Dropout(rate=0.2)(lstm_output)
#     flatten_output = layers.Flatten()(dropout_output)
#     dense_output1 = layers.Dense(64, activation='relu')(flatten_output)
#     dropout_output2 = layers.Dropout(rate=0.2)(dense_output1)
#     dense_output2 = layers.Dense(4, activation='softmax')(dropout_output2)
#     lstm_classifier = Model(model_input, dense_output2)
#     #lstm_classifier.summary()
#     lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(1e-3))

#     return lstm_classifier

sx_model = buildModelv2(timestep, sx_n_gestures)
dx_model = buildModelv2(timestep, dx_n_gestures)
dx_model.summary()

In [ ]:
# CHANGE NAME EVERY TIME

# sx_mc = tfk.callbacks.ModelCheckpoint('/home/npasini1/Desktop/dVRK_UserStudy/model_checkpoints/UserStudy_sx_5_ALL.h5',
#                                         monitor='loss',
#                                         mode = 'min',
#                                         save_best_only=True,
#                                         verbose=1)

# dx_mc = tfk.callbacks.ModelCheckpoint('/home/npasini1/Desktop/dVRK_UserStudy/model_checkpoints/UserStudy_dx_5_ALL.h5',
#                                         monitor='loss',
#                                         mode = 'min',
#                                         save_best_only=True,
#                                         verbose=1)

In [ ]:
batch_size = 256
for i in range(9):
    sx_model = buildModelv2(timestep, sx_n_gestures)
    tfk.backend.clear_session()
    user = users[i]
    sx_mc = tfk.callbacks.ModelCheckpoint('/home/npasini1/Desktop/dVRK_UserStudy/model_checkpoints/batch256/' + user + '_sx_5_256.h5',
                                        monitor='loss',
                                        mode = 'min',
                                        save_best_only=True,
                                        verbose=1)
    sx_data = users_dataset_sx[i]
    sx_labels = sx_encoded_list[i]
    history = sx_model.fit(
        # x = sx_datadataset_tot,
        x = sx_data,
        # y = sx_encoded,
        y = sx_labels,
        batch_size = batch_size,
        epochs = 2000,
        # validation_split=.20,
        #validation_steps=10,
        shuffle=False,
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='loss', mode='min', patience=210),
            tfk.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', patience=40, factor=0.5, min_lr=1e-5),
            sx_mc
        ]
    ).history

In [ ]:
batch_size = 256

for i in range(9):
    dx_model = buildModelv2(timestep, dx_n_gestures)
    tfk.backend.clear_session()
    user = users[i]
    dx_mc = tfk.callbacks.ModelCheckpoint('/home/npasini1/Desktop/dVRK_UserStudy/model_checkpoints/batch256/' + user + '_dx_5_256.h5',
                                        monitor='loss',
                                        mode = 'min',
                                        save_best_only=True,
                                        verbose=1)
    dx_data = users_dataset_dx[i]
    dx_labels = dx_encoded_list[i]
    history = dx_model.fit(
        # x = dx_dataset_tot,
        x = dx_data,
        # y = dx_encoded,
        y = dx_labels,
        batch_size = batch_size,
        epochs = 2000,
        # validation_split=.20,
        #validation_steps=10,
        shuffle=False,
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='loss', mode='min', patience=210),
            tfk.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', patience=40, factor=0.5, min_lr=1e-5),
            dx_mc
        ]
    ).history

In [ ]:
# sx_model.save('/home/npasini1/Desktop/dVRK_UserStudy/model_save/UserStudy_sx_5_modelsave_ALL.h5')
# dx_model.save('/home/npasini1/Desktop/dVRK_UserStudy/model_save/UserStudy_dx_5_modelsave_ALL.h5')

In [ ]:
# sx_model.evaluate(sx_X_test, sx_y_test)
# dx_model.evaluate(dx_X_test, dx_y_test)

In [ ]:
# model.predict(np.expand_dims(X_test[0],0))
# def a():
#     prediction = model.predict(np.expand_dims(X_RealTime,axis=0), verbose=0)

In [ ]:
# model.predict(np.expand_dims(X_test[34],0))

In [ ]:
# X_test[0].shape

In [ ]:
# model = tfk.models.load_model('/home/npasini1/nicolo_ws/src/pyrecorder/src/bestmodel_v2_senzapreprocessing.h5')

In [ ]:
# model = tfk.models.load_model('/home/npasini1/Desktop/model_checkpoints/magnet.h5')

In [ ]:
# a = model.predict(X_test)
# print(X_test.shape)
# prediction = np.argmax(a, axis=1)
# print(prediction)